In [40]:
import torch
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import matplotlib.pyplot as plt

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from matplotlib import pyplot as plt

We download the data in the same way we did when using ````sklearn```` library.

Note that here we also chagne the **data type** to ````torch.tensor````, because we want to use functions from the ````pytorch```` library.

In [41]:
from sklearn import datasets
digits = datasets.load_digits()
img = digits.images
labels = digits.target

X = img.reshape((len(img),-1))
from sklearn.model_selection import train_test_split
test_size=0.2
X_train, X_test, y_train, y_test = train_test_split(
    X, labels, test_size = test_size, shuffle=False)

X_train = X_train.astype(np.float32)
X_train = torch.tensor(X_train)
X_test = torch.tensor(X_test)
y_train = torch.tensor(y_train)
y_train=y_train.long()
MLPClassifiery_test = torch.tensor(y_test)

### Q1 

Let's build a ````Network```` class. This will be your model.

You will need
* ````__init__```` function defining the attributes. These attributes should be the different layers of your model.
* You can define a linear (fully-connected) layer using ````Linear(input_size,output_size)```` function from "torch.nn" library (imported as "nn")
* You can define an activation function using ````ReLU()```` function, also from "torch.nn" library
* ````forward```` function, defining how the input is processed in the forward pass, layer by layer.

In [42]:

class Network(nn.Module):
    def __init__(self,input_size,output_size):
        super(Network, self).__init__()
        self.linear_layer=nn.Linear(input_size,output_size)
        self.relu=nn.ReLU()
    def forward(self,x):
        out=self.linear_layer(x)
        out=self.relu(out)
        return out

### Q2
Define your model as instance of your Network class.

In [43]:
model=Network(64,32)

Below we define the optimized and loss function.

*For now you can just use this as it is. As you learn more about Machine Learning, you might want to experiment with changing different parameters in this setup, or even picking a different loss function and a better optimizer.*

*It takes a lot of experience to learn all bits and pieces of this machinery, and all programmers spend a long time just copy+paste-ing this default setup to their code.*

*The rule of thumb is: if it works well, let's use it and see what happens!*

In [44]:
learning_rate =0.001
epochs = 200
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
loss_func = nn.CrossEntropyLoss()

### Q3
Model training.

You will train your model in a loop. Loop should be over epochs ("training epochs"). We will do this step by step.

In [45]:
def how_loss_changes_as_epoch_changes(epoch):
    storage_of_loss=[]
    for e in range(epoch):

        ## first, at the beginning of the loop, make sure that the optimizer kills all previous grad values to avoid using them again in your new iteration
        optimizer.zero_grad()

        # now, perform the forward step by applying your model to your dataset
        output = model.forward(X_train)

        # calculate the loss, using the "loss_func" applied to the output from your model compared with the real labels
        loss =loss_func(output,y_train)
        l='{:.6f}'.format(loss)
        m=float(l)
        storage_of_loss.append(m)
        # MAGIC time: perform the back-prop:
        loss.backward()
        optimizer.step()
        # at each 10 steps, let's print the value of the loss to see if it is changing.
        if e%10==0:
            print('Epoch: {} - Loss: {:.6f}'.format(e, loss))
    print("all the loss are saved",storage_of_loss)
    return m

In [46]:
how_loss_changes_as_epoch_changes(200)

Epoch: 0 - Loss: 8.152740
Epoch: 10 - Loss: 3.893719
Epoch: 20 - Loss: 1.970841
Epoch: 30 - Loss: 1.375838
Epoch: 40 - Loss: 1.000165
Epoch: 50 - Loss: 0.603992
Epoch: 60 - Loss: 0.367051
Epoch: 70 - Loss: 0.258301
Epoch: 80 - Loss: 0.204968
Epoch: 90 - Loss: 0.176085
Epoch: 100 - Loss: 0.158929
Epoch: 110 - Loss: 0.147239
Epoch: 120 - Loss: 0.138468
Epoch: 130 - Loss: 0.131443
Epoch: 140 - Loss: 0.125570
Epoch: 150 - Loss: 0.120508
Epoch: 160 - Loss: 0.116051
Epoch: 170 - Loss: 0.112065
Epoch: 180 - Loss: 0.108460
Epoch: 190 - Loss: 0.105168
all the loss are saved [8.15274, 7.718696, 6.994428, 6.156381, 5.409637, 4.926834, 4.690975, 4.525872, 4.339681, 4.130936, 3.893719, 3.591915, 3.243613, 2.97013, 2.846514, 2.797245, 2.718455, 2.568877, 2.362587, 2.145787, 1.970841, 1.864849, 1.815883, 1.791278, 1.759325, 1.702216, 1.621982, 1.536011, 1.463062, 1.410712, 1.375838, 1.349532, 1.322788, 1.292095, 1.255291, 1.212901, 1.169037, 1.12322, 1.077273, 1.035271, 1.000165, 0.973957, 0.950731, 

0.102434

In [47]:
e=np.linspace(0,200,201)
losses=how_loss_changes_as_epoch_changes(e)
plt.plot(e,losses)

TypeError: only integer scalar arrays can be converted to a scalar index

### Q4
Modify your code so that
* It only prints the value of the loss every 10 steps
* saves down values of loss and values of accuracy at each epoch (each step)

Then, plot on separate graphs 
* the accuracy as function of epochs
* the loss as function of epochs

Is the model training well?